<a href="https://colab.research.google.com/github/kerryback/Classic_Tests/blob/main/Fama_MacBeth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Overview

We're going to form portfolios based on rolling-window beta estimates.  Then, we we will estimate the portfolio betas and residual standard deviations, again using rolling-window regressions. Then, we will run regressions on the cross-sections of portfolios to determine whether beta, squared beta, and residual standard deviation predict returns.  This is the Fama-MacBeth test of the CAPM.

In [1]:
!pip install --upgrade statsmodels

     |████████████████████████████████| 9.8 MB 51.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

# Read the file created and saved in the Black_Jensen_Scholes notebook.

In [3]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/crsp_compustat_example2.csv', parse_dates=['date'])
df.date = df.date.dt.to_period('M')
df = df.sort_values(by=['permno','date'])

Mounted at /content/drive


# Sort into 20 portfolios each month and compute value-weighted returns

In [4]:
df['group'] = df.groupby('date').beta.apply(lambda x: pd.qcut(x,20,labels=range(1,21)))
rets = df.groupby(['date','group']).apply(lambda d: (d.me*d.ret).sum() / d.me.sum())
rets = pd.DataFrame(rets).reset_index()
rets.columns = ['date','group','ret']
df = df.drop(columns=['group'])

# Merge market excess return and risk-free rate

In [5]:
ff = pdr('F-F_Research_Data_Factors','famafrench',start='1962-01-01')[0] / 100
rets = rets.merge(ff[['Mkt-RF','RF']],left_on='date',right_index=True,how='left')

# Rolling betas and idiosyncratic volatilities of portfolios

Run rolling window regressions and keep betas and residual standard deviations.

In [6]:
def rollingReg(d) :
    try :
        result = RollingOLS(d.ret-d.RF,sm.add_constant(d['Mkt-RF']),window=60,min_nobs=60).fit()
        df = pd.DataFrame(result.params['Mkt-RF'])
        df.columns = ['beta']
        df['rstd'] = np.sqrt(result.mse_resid)
        df.index = d.date
        return df
    except :
        return pd.DataFrame(np.nan,index=d.date,columns=['beta'])
data = rets.groupby('group').apply(rollingReg)
rets = rets.merge(data,left_on=['group','date'],right_index=True,how='inner')
rets = rets.dropna(subset=['beta','rstd','ret'],how='any')

# Fama-MacBeth regressions

Run a cross-sectional regression (cross-section = 20 portfolios) each month on beta, squared beta, and residual standard deviation:

$$r_{pt} = \alpha_t + \gamma_{1t} \hat{\beta}_{pt} + \gamma_{2t}\hat{\beta}_{pt}^2 + \gamma_{3t}\hat{\sigma}_{pt} + e_{pt}$$

In [7]:
rets['beta2'] = rets['beta']**2
coefs = rets.groupby('date').apply(lambda d: sm.OLS(d.ret,sm.add_constant(d[['beta','beta2','rstd']])).fit().params)  

# Test the CAPM

According to the CAPM, the coefficient on $\beta$ should be the market risk premium, and the coefficients on $\beta^2$ and the residual standard deviation should be zero.  We run $t$ tests of the nulls that the coefficients are zero.  We cannot reject any of the null hypotheses.

In [8]:
from scipy.stats import ttest_1samp as ttest

stats = pd.DataFrame(dtype=float,index=coefs.columns,columns=['tstat','pval'])
for coef in coefs.columns :
    t, p = ttest(coefs[coef],0)
    stats.loc[coef,'tstat'] = t
    stats.loc[coef,'pval'] = p
stats.round(3)

,tstat,pval
const,2.707,0.007
beta,-0.788,0.431
beta2,0.944,0.345
rstd,-0.791,0.429
